In [ ]:
pip install /clip/Samuel_workdir/git/CellCNN

In [ ]:
from CellCNN import *
import matplotlib.pyplot as plt

In [ ]:
from CellCNN import *
d1 = Dataset(10000, center_density=0.9)
d2 = Dataset(10000, center_density=0.5)
d3 = Dataset(10000, center_density=0.1)
ID = InputData((d1,d2,d3), labels = np.array(([0],[1], [2])))
Xt, Yt = ID.get_multi_cell_inputs(5000)
Xv, Yv = ID.get_multi_cell_inputs(1000)

In [7]:
m = CellCNN((None, 1000, 2), conv=[64,64,16], l1_weight=0, classes = [-3])
m.run_eagerly=False
x = 1
# m.init_random(Xt[:10])
for j in range(x):
    print(j)
    # Xtb = Xt[Xt.shape[0]//x*j:Xt.shape[0]//x*(j+1)]
    # Ytb = [Yt[i][Yt[i].shape[0]//x*j:Yt[i].shape[0]//x*(j+1)] for i in range(1)]
    m.fit(Xt,Yt, validation_data=(Xv,Yv), epochs = 10)
    sm = m.get_single_cell_model()
    cells = Xt.reshape(-1,2)[:10000]
    res = sm(cells)
for i in range(16):
    if(tf.reduce_mean(res, 0)[i]>0):
        print(f"filter: {i}")
        plt.scatter(cells[:,0], cells[:,1], c=res[:,i])
        plt.show()
    else:
        print(f"filter {i} SKIPPED")

In [ ]:
y_true=tf.constant((1,0,2)),
y_pred = tf.constant(((0.,1.,0.),(0.,0.,1.),(0.,0.,1.))),
mask = K.cast(K.not_equal(y_true, -1), "bool")
y_true_masked = tf.boolean_mask(y_true,mask)
y_pred_masked = tf.boolean_mask(y_pred,mask)
cdf_true = K.cumsum(tf.one_hot(tf.cast(y_true_masked, "int32"), y_pred.shape[1]), axis=-1)
cdf_pred = K.cumsum(y_pred_masked, axis=-1)
emd = K.sqrt(K.mean(K.square(cdf_true - cdf_pred), axis=-1))

In [ ]:
cells = Xv.reshape(3000000,2)
res = sm(cells)
print(tf.reduce_mean(res, 0))

In [ ]:
sm = m.get_single_cell_model()
cells = Xv.reshape(3000000,2)[:10000]
res = sm(cells)
print(tf.reduce_mean(res, 0))

In [ ]:

for i in range(16):
    plt.scatter(cells[:,0], cells[:,1], c=res[:,i])
    plt.show()

In [ ]:
from CellCNN import *
import matplotlib.pyplot as plt
import numpy as np

from CellCNN import *
d1 = Dataset(100000, center_density=0.9)
d2 = Dataset(100000, center_density=0.1)
d1.data += 2
ID = InputData((d1,d2), labels = np.array(([1,5,2],[0,10,1])))
Xt, Yt = ID.get_multi_cell_inputs(12000)
Xv, Yv = ID.get_multi_cell_inputs(3000)

print(np.sum(res!=0,0)/res.shape[0]*100)
print(res.shape)

m.loss[0]

done=False
while not done:
    m = CellCNN((None, 1000, 2), conv=[64,64,16], l1_weight=1e-11, classes = [2,0,3])
    x = 10
    epochs = 1
    m.init_random(Xt[:10])
    values = []
    best_loss = float(0)
    for j in range(x):
        print(j)
        perm = np.random.permutation(Xt.shape[0])
        Xtb = Xt[perm]
        Ytb = [Yt[i][perm] for i in range(3)]
        cb = m.fit(Xtb,Ytb, validation_data=(Xv,Yv), epochs = epochs)
        sm = m.get_single_cell_model()
        # cells = Xt.reshape(-1,2)
        cells = Xv.reshape(Xv.shape[0]*Xv.shape[1],2)[:5000]
        res = sm(cells)
        values.append(tf.reduce_mean(res, 0))
        if cb.history["val_output_1_accuracy"][-1] > 0.8:
            best_loss = cb.history["val_output_1_accuracy"][-1]
            best_weights = m.get_weights()
            best_j = j
            done = True
            break
print(f"best_j = {best_j}")
m.set_weights(best_weights)
for i in range(16):
    if np.sum(res[:,i])>0:
        print(f"Filter {i}:")
        plt.scatter(cells[:,0], cells[:,1], c=res[:,i])
        plt.show()
    else:
        print(f"Filter {i} SKIPPED.")

cells = Xt.reshape(Xt.shape[0]*Xt.shape[1],2)[:5000]
res = sm(cells)
for i in range(16):
    if np.sum(res[:,i])>0:
        print(f"Filter {i}:")
        plt.scatter(cells[:,0], cells[:,1], c=res[:,i])
        plt.show()
    else:
        print(f"Filter {i} SKIPPED.")

for j in range(x):
    print(j)
    Xtb = Xt[Xt.shape[0]//x*j:Xt.shape[0]//x*(j+1)]
    Ytb = [Yt[i][Yt[i].shape[0]//x*j:Yt[i].shape[0]//x*(j+1)] for i in range(1)]
    cb = m.fit(Xtb,Ytb, validation_data=(Xv,Yv), epochs = 1)
    sm = m.get_single_cell_model()
    # cells = Xt.reshape(-1,2)
    cells = Xv.reshape(3000000,2)[:5000]
    res = sm(cells)
    values.append(tf.reduce_mean(res, 0))
    if cb.history["val_loss"][-1] < best_loss:
        best_loss = cb.history["val_loss"][-1]
        best_weights = m.get_weights()
        best_j = j
m.set_weights(best_weights)
plt.plot(values)
plt.legend(list(range(16)), loc='upper right', bbox_to_anchor=(1.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
print(values[-1])
plt.show()

for i in range(16):
    if np.sum(res[:,i])>0:
        print(f"Filter {i}:")
        plt.scatter(cells[:,0], cells[:,1], c=res[:,i])
        plt.show()
    else:
        print(f"Filter {i} SKIPPED.")

cells = Xv.reshape(3000000,2)
res = sm(cells)
print(tf.reduce_mean(res, 0))

sm = m.get_single_cell_model()
cells = Xv.reshape(3000000,2)[:5000]
res = sm(cells)
print(tf.reduce_mean(res, 0))

for i in range(16):
    plt.scatter(cells[:,0], cells[:,1], c=res[:,i])
    plt.show()

x = tf.constant(3.0)
with tf.GradientTape() as g:
  y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

In [ ]:
y_true = tf.constant((1,0,1,2,1,2,-1))[:,None]
y_pred = tf.constant(((0.,0.9,0.1),
            (0.9,0.1,0.),
            (0.,0.9,0.1),
            (0.,0.1,0.9),
            (0.,0.9,0.1),
            (0.,0.1,0.9),
            (0.9,0.1,0.)))
mask = K.cast(K.not_equal(y_true, -1), "bool")
y_true = K.cast(y_true, K.floatx())
y_pred = K.cast(y_pred, K.floatx())

y_true_masked = tf.ragged.boolean_mask(y_true, mask[:,0])
y_pred_masked = tf.ragged.boolean_mask(y_pred, mask[:,0])
print( tf.keras.losses.sparse_categorical_crossentropy(y_true_masked, y_pred_masked))

In [ ]:
tf.constant(((0.,0.9,0.1),
(0.9,0.1,0.),
(0.,0.9,0.1),
(0.,0.1,0.9),
(0.,0.9,0.1),
(0.,0.1,0.9),
(0.9,0.1,0.)))
